In [2]:
HOME = "./TextData/"

Mounted at /content/drive


# Install dependencies

In [3]:

!pip install datasets==2.16.0
!pip install huggingface-hub==0.20.1


!pip install python-dateutil==2.8.2
!pip install pytz==2023.3.post1
!pip install PyYAML==6.0.1
!pip install regex==2023.12.25

!pip install safetensors==0.4.1
!pip install scikit-learn==1.3.2
#!pip install setuptools==68.2.2

!pip install tokenizers==0.13.3
!pip install torch==2.1.0
!pip install tqdm==4.66.1
!pip install transformers==4.28.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.3
    Uninstalling huggingface-hub-0.23.3:
      Successfully uninstalled huggingface-hub-0.23.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency 

# Imports

In [4]:
import numpy as np
import torch
import torch.distributed as dist
from packaging import version
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score
from torch import nn
from torch.func import functional_call, jvp
from torch.optim import SGD, Adam, Adagrad
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.auto import tqdm
from transformers import Trainer
from transformers.debug_utils import DebugOption, DebugUnderflowOverflow
from transformers.deepspeed import deepspeed_init, is_deepspeed_zero3_enabled
from transformers.dependency_versions_check import dep_version_check
# Integrations must be imported before ML frameworks:
from transformers.integrations import (  # isort: split
    hp_params,
    is_fairscale_available,
)
from transformers.pytorch_utils import is_torch_greater_or_equal_than_1_10, is_torch_less_than_1_11
from transformers.trainer_callback import (
    DefaultFlowCallback,
    ProgressCallback,
    TrainerState,
)
from transformers.trainer_pt_utils import (
    IterableDatasetShard,
)
from transformers.trainer_utils import (
    HPSearchBackend,
    ShardedDDPOption,
    TrainOutput,
    has_length,
    speed_metrics,
)
from transformers.utils import (
    WEIGHTS_NAME,
    is_apex_available,
    is_in_notebook,
    is_sagemaker_mp_enabled,
    is_torch_tpu_available,
    logging,
)

from torch.nn import CrossEntropyLoss

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    HfArgumentParser,
    TrainingArguments,
    DataCollatorForTokenClassification,
    AutoModelForSequenceClassification
)

from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# from torch.optim.optimizer import StateDict, params_t

from torch.func import functional_call, jvp

import torch.nn.functional as F
import math
import time
from functools import partial

# Utility Functions

In [5]:
def set_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

In [6]:
def exponential_lr_decay(step: int, k: float=1e-4):
    return math.e ** (-step * k)

# Models

In [7]:
def load_model(args):
  config = AutoConfig.from_pretrained(args.model_name)
  model = AutoModelForSequenceClassification.from_pretrained(args.model_name, config=config, torch_dtype=torch.float32)
  tokenizer = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)

  return model, tokenizer

# DataSet

In [8]:
class MovieReviewDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len



  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
    review,
    add_special_tokens=True,
    max_length=self.max_len,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
    truncation = True
  )
    return {
    'input_ids': encoding['input_ids'].flatten(),
    'attention_mask': encoding['attention_mask'].flatten(),
    'labels': torch.tensor(target, dtype=torch.long)
  }







In [9]:
def to_sentiment(rating):
  rating = str(rating)
  if rating == 'positive':
    return 0
  else:
    return 1

def get_data(path: str, tokenizer, args):
  data = pd.read_csv(path)
  data = data[:4000]
  data['sentiment_score'] = data.sentiment.apply(to_sentiment)

  df_train, df_test = train_test_split(data, test_size=0.3, random_state=123)
  df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=123)

  df_train_ds = MovieReviewDataset(
    reviews=df_train.review.to_numpy(),
    targets=df_train.sentiment_score.to_numpy(),
    tokenizer=tokenizer,
    max_len=args.max_len
  )


  df_test_ds = MovieReviewDataset(
    reviews=df_test.review.to_numpy(),
    targets=df_test.sentiment_score.to_numpy(),
    tokenizer=tokenizer,
    max_len=args.max_len
  )

  df_val_ds = MovieReviewDataset(
    reviews=df_val.review.to_numpy(),
    targets=df_val.sentiment_score.to_numpy(),
    tokenizer=tokenizer,
    max_len=args.max_len
  )

  train_data_loader = DataLoader(
    df_train_ds,
    batch_size=args.batch_size,
    num_workers=4
  )
  val_data_loader = DataLoader(
    df_val_ds,
    batch_size=args.batch_size,
    num_workers=4
  )
  test_data_loader = DataLoader(
    df_test_ds,
    batch_size=args.batch_size,
    num_workers=4
  )

  return train_data_loader, test_data_loader, val_data_loader

# Train Functions

In [10]:
class MyArguments():
  def __init__(self, epochs=4, max_len=400, batch_size=16, optimizer="adam", learning_rate=1e-5, trainer='forward_grad', model_name='', max_length=2048, max_grad_norm=0, momentum = 0.0):
    self.epochs = epochs
    self.max_len = max_len
    self.batch_size = batch_size
    self.optimizer= optimizer
    self.learning_rate = learning_rate
    self.trainer=trainer
    self.model_name = model_name
    self.max_length = max_length
    self.weight_decay = 0
    self.max_grad_norm = max_grad_norm
    self.momentum = momentum


In [11]:
class MyTrainer():
  def __init__(self, model, args, train_loader, test_loader, val_loader):
    self.model = model
    self.args = args
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.val_loader = val_loader

    use_cuda = torch.cuda.is_available()
    self.device = torch.device("cuda:0" if use_cuda else "cpu")

  def backward_grad_step(self, inputs):
    inputs = {k: v.to(device=self.device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

    if "labels" in inputs:
        labels = inputs.pop("labels")
    else:
        labels = None

    y = self.model(**inputs).logits
    loss = F.cross_entropy(y, labels)
    loss.backward()

    return loss

  def _get_learning_rate(self):
    for param_group in self.optimizer.param_groups:
      return param_group['lr']

  @staticmethod
  @torch.no_grad()
  def functional_call_loss(params, names, buffers, model, batch, t: torch.Tensor):
        model_params = {k: v for k, v in zip(names, params)}

        outputs = functional_call(model, (model_params, buffers), tuple(), kwargs=batch)
        logits = outputs.logits

        return F.cross_entropy(logits, t)


  def forward_grad_step(self, inputs):
    with torch.no_grad():
      named_buffers = dict(self.model.named_buffers())
      named_params = dict(self.model.named_parameters())
      names = named_params.keys()
      params = named_params.values()

      v_params = tuple([torch.randn_like(p) for p in params])
      inputs = {
        k: v.to(device=self.device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()
      }

      if "labels" in inputs:
          labels = inputs.pop("labels")
      else:
          labels = None
      f = partial(
                self.functional_call_loss,
                names=names,
                buffers=named_buffers,
                model=self.model,
                batch=inputs,
                t=labels
      )

      # Forward AD
      loss, jvp_ = jvp(f, (tuple(params),), (v_params,))

      # Setting gradients
      for v, p in zip(v_params, params):
          p.grad = v * jvp_


      return loss




  def train(self, name):
    rows_list = []
    validation_row_list = []
    performance_row_list = []


    self.model.to(self.device)
    self.model.float()
    self.model.train()

    if self.args.optimizer == "adam":
      self.optimizer = Adam(self.model.parameters(), lr=self.args.learning_rate)
    elif self.args.optimizer == "sgd":
      self.optimizer = SGD(self.model.parameters(), lr=self.args.learning_rate)
    elif self.args.optimizer == "adagrad":
      self.optimizer = torch.optim.Adagrad(self.model.parameters(), lr=args.learning_rate)
    elif self.args.optimizer == "momentumSGD":
      self.optimizer = SGD(self.model.parameters(), lr=self.args.learning_rate, momentum=self.args.momentum)

    self.optimizer.zero_grad(set_to_none=True)
    self.scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=self.optimizer, lr_lambda=exponential_lr_decay)



    # Train
    steps = 0
    t_total = 0.0
    validation_steps = 0

    for epoch in range(self.args.epochs):
        t0 = time.perf_counter()
        loss = 0
        with tqdm(total=len(self.train_loader)) as pbar:
          for batch in self.train_loader:
              pbar.update(1)
              steps += 1

              if self.args.trainer == "forward_grad":
                loss_step = self.forward_grad_step(batch)
              else:
                loss_step = self.backward_grad_step(batch)
              loss += loss_step

              rows_list.append({
                  "step": steps,
                  "loss_step": loss_step.item()
              })

              if self.args.max_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)

              # Optimizer step
              self.optimizer.step()

              # Lr scaling
              self.scheduler.step()

              # Zero out grads
              self.optimizer.zero_grad(set_to_none=True)

        t1 = time.perf_counter()
        t_total += t1 - t0
        print("Time/batch_time", t1 - t0, steps)
        print("Time/sps", steps / t_total, steps)

        acc = 0
        all_pred = []
        all_true = []
        with torch.no_grad():
          for batch in self.val_loader:
              inputs = {
                k: v.to(device=self.device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()
              }

              validation_steps += 1


              if "labels" in inputs:
                labels = batch.pop("labels")
              else:
                labels = None

              out = self.model(**inputs)
              logits = out.logits
              pred = F.softmax(logits, dim=-1).argmax(dim=-1)



              labels=labels.to(device=self.device)

              validation_row_list.append({
                  "step": validation_steps,
                  "loss_step": F.cross_entropy(logits, labels).item()
              })

              acc += (pred == labels).sum()
              all_pred.extend(pred.cpu())
              all_true.extend(labels.cpu())

        f1 = f1_score(all_true, all_pred)
        performance_row_list.append({
            "accuracy": (acc / len(self.val_loader.dataset)).item(),
            "f1_score": f1,
            "epoch": epoch+1

        })
        print(f"Epoch [{epoch+1}/{self.args.epochs}], Loss: {loss.item():.4f}, Time (s): {t1 - t0:.4f}, Test accuracy: {(acc / len(self.val_loader.dataset)).item():.4f} F1: {f1}")
    print(f"Mean time: {t_total / self.args.epochs:.4f}")

    # Test
    acc = 0
    all_pred = []
    all_true = []
    with torch.no_grad():
      for batch in self.test_loader:
          inputs = {
                  k: v.to(device=self.device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()
          }
          if "labels" in inputs:
            labels = batch.pop("labels")
          else:
            labels = None
          labels=labels.to(device=self.device)

          out = self.model(**inputs).logits
          pred = F.softmax(out, dim=-1).argmax(dim=-1)

          acc += (pred == labels).sum()
          all_pred.extend(pred.cpu())
          all_true.extend(labels.cpu())
    f1 = f1_score(all_true, all_pred)
    print()
    print( "------- Test/accuracy", acc / len(self.test_loader.dataset), steps)
    print(f"------- Test accuracy: {(acc / len(self.test_loader.dataset)).item():.4f}")
    print(f"------- Test F1: {f1}")

    train_pd = pd.DataFrame(rows_list)
    validation_pd = pd.DataFrame(validation_row_list)
    performance_pd = pd.DataFrame(performance_row_list)

    train_pd.to_csv(HOME + name + "_train_loss.csv")
    validation_pd.to_csv(HOME + name + "_validation_loss.csv")
    performance_pd.to_csv(HOME + name + "_performance_metrics.csv")

# Train

In [12]:
set_seed(32)

## Forward Gradiend

### Adam

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="adam",
    learning_rate=1e-5,
    trainer='forward_grad',
    model_name="FacebookAI/roberta-base"
)

In [ ]:
model, tokenizer = load_model(args)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("adam_forward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 245.80276309699912 175
Time/sps 0.71195294062232 175
Epoch [1/10], Loss: 121.5476, Time (s): 245.8028, Test accuracy: 0.4783 F1: 0.31808278867102396


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 244.87855190000118 1750
Time/sps 0.7146272831533993 1750
Epoch [10/10], Loss: 121.7100, Time (s): 244.8786, Test accuracy: 0.4583 F1: 0.3243243243243243
Mean time: 244.8829


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.4683, device='cuda:0') 1750
------- Test accuracy: 0.4683
------- Test F1: 0.2989010989010989


### Adagrad

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="adagrad",
    learning_rate=1e-5,
    trainer='forward_grad',
    model_name="FacebookAI/roberta-base"
)

In [ ]:
model, tokenizer = load_model(args)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("adagrad_forward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 243.11861269000292 175
Time/sps 0.7198132551995927 175
Epoch [1/10], Loss: 121.7590, Time (s): 243.1186, Test accuracy: 0.4833 F1: 0.07185628742514971


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 243.24333950800064 1750
Time/sps 0.7200600229549372 1750
Epoch [10/10], Loss: 121.9514, Time (s): 243.2433, Test accuracy: 0.4750 F1: 0.059701492537313425
Mean time: 243.0353


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.5050, device='cuda:0') 1750
------- Test accuracy: 0.5050
------- Test F1: 0.06309148264984227


### Clipped SGD

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="sgd",
    learning_rate=1e-5,
    trainer='forward_grad',
    model_name="FacebookAI/roberta-base",
    max_grad_norm=1
)
model, tokenizer = load_model(args)
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("sgd_forward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 238.0406420979998 175
Time/sps 0.7351685764986032 175
Epoch [1/10], Loss: 121.6390, Time (s): 238.0406, Test accuracy: 0.4867 F1: 0.2631578947368421


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 237.7288161960023 1750
Time/sps 0.7348816738933497 1750
Epoch [10/10], Loss: 121.9710, Time (s): 237.7288, Test accuracy: 0.5183 F1: 0.3167848699763593
Mean time: 238.1336


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.4867, device='cuda:0') 1750
------- Test accuracy: 0.4867
------- Test F1: 0.23


### Momentum SGD

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="momentumSGD",
    learning_rate=1e-5,
    trainer='forward_grad',
    model_name="FacebookAI/roberta-base",
    momentum=0.9
)
model, tokenizer = load_model(args)
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("momentumSGD_forward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 229.33362633500656 175
Time/sps 0.7630804204193024 175
Epoch [1/10], Loss: 121.5540, Time (s): 229.3336, Test accuracy: 0.4783 F1: 0.31509846827133475


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 241.25841976200172 1750
Time/sps 0.7339334767010439 1750
Epoch [10/10], Loss: 121.7400, Time (s): 241.2584, Test accuracy: 0.4500 F1: 0.2600896860986547
Mean time: 238.4412


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.4817, device='cuda:0') 1750
------- Test accuracy: 0.4817
------- Test F1: 0.2817551963048499


### SGD

In [26]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="sgd",
    learning_rate=1e-3,
    trainer='forward_grad',
    model_name="FacebookAI/roberta-base",
    max_grad_norm=1
)
model, tokenizer = load_model(args)
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [27]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("sgd_forward_lr3")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 249.53872042700004 175
Time/sps 0.7012939703327301 175
Epoch [1/10], Loss: 121.8505, Time (s): 249.5387, Test accuracy: 0.4667 F1: 0.2488262910798122


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 249.06689562800057 1750
Time/sps 0.7031987063100658 1750
Epoch [10/10], Loss: 121.7894, Time (s): 249.0669, Test accuracy: 0.4883 F1: 0.2974828375286041
Mean time: 248.8628


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.4750, device='cuda:0') 1750
------- Test accuracy: 0.4750
------- Test F1: 0.2622950819672131


## Backward

### Adam

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="adam",
    learning_rate=1e-5,
    trainer='backward',
    model_name="FacebookAI/roberta-base"
)

In [ ]:
model, tokenizer = load_model(args)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

<ipython-input-7-bbbcf39e7877>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment_score'] = data.sentiment.apply(to_sentiment)


In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("adam_backward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 196.28936996899938 175
Time/sps 0.8915408920393311 175
Epoch [1/10], Loss: 68.2536, Time (s): 196.2894, Test accuracy: 0.8933 F1: 0.8926174496644296


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 195.87933271099973 1750
Time/sps 0.8937843800456687 1750
Epoch [10/10], Loss: 0.5079, Time (s): 195.8793, Test accuracy: 0.9333 F1: 0.9395770392749246
Mean time: 195.7967


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.9267, device='cuda:0') 1750
------- Test accuracy: 0.9267
------- Test F1: 0.9310344827586207


### Adagrad

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="adagrad",
    learning_rate=1e-5,
    trainer='backward',
    model_name="FacebookAI/roberta-base"
)

model, tokenizer = load_model(args)
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("adagrad_backward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 195.63843879399792 175
Time/sps 0.8945072403908843 175
Epoch [1/10], Loss: 120.7808, Time (s): 195.6384, Test accuracy: 0.6067 F1: 0.6143790849673202


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 193.8340595209993 1750
Time/sps 0.9007812728547223 1750
Epoch [10/10], Loss: 31.0501, Time (s): 193.8341, Test accuracy: 0.9450 F1: 0.9489953632148377
Mean time: 194.2758


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.9417, device='cuda:0') 1750
------- Test accuracy: 0.9417
------- Test F1: 0.9430894308943089


### Clipped SGD

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="sgd",
    learning_rate=1e-5,
    trainer='backward',
    model_name="FacebookAI/roberta-base",
    max_grad_norm=1
)

model, tokenizer = load_model(args)
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("sgd_backward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 190.22871404200123 175
Time/sps 0.9199452400301732 175
Epoch [1/10], Loss: 121.8041, Time (s): 190.2287, Test accuracy: 0.4700 F1: 0.13114754098360656


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 189.00686238599883 1750
Time/sps 0.9260724248737371 1750
Epoch [10/10], Loss: 121.5426, Time (s): 189.0069, Test accuracy: 0.4667 F1: 0.19999999999999998
Mean time: 188.9701


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.4867, device='cuda:0') 1750
------- Test accuracy: 0.4867
------- Test F1: 0.23762376237623759


### MomentumSGD

In [ ]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="momentumSGD",
    learning_rate=1e-5,
    trainer='backward',
    model_name="FacebookAI/roberta-base",
    momentum=0.9
)

model, tokenizer = load_model(args)
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("momentumSGD_backward")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 190.34832497099706 175
Time/sps 0.9193671655721917 175
Epoch [1/10], Loss: 121.6712, Time (s): 190.3483, Test accuracy: 0.4867 F1: 0.1348314606741573


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 190.8938960919986 1750
Time/sps 0.9175803704926773 1750
Epoch [10/10], Loss: 121.1467, Time (s): 190.8939, Test accuracy: 0.4883 F1: 0.4752136752136752
Mean time: 190.7190


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.5317, device='cuda:0') 1750
------- Test accuracy: 0.5317
------- Test F1: 0.5044091710758377


### SGD

In [13]:
args = MyArguments(
    epochs=10,
    max_len=400,
    batch_size=16,
    optimizer="sgd",
    learning_rate=1e-3,
    trainer='backward',
    model_name="FacebookAI/roberta-base"
)

model, tokenizer = load_model(args)
loaders = get_data(HOME +  "IMDB_Dataset.csv", tokenizer, args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_pro

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

<ipython-input-9-bbbcf39e7877>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment_score'] = data.sentiment.apply(to_sentiment)


In [14]:
trainer = MyTrainer(model, args, loaders[0], loaders[1], loaders[2])
trainer.train("sgd_backward_lr3")

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 189.583307129 175
Time/sps 0.9230770506652417 175
Epoch [1/10], Loss: 121.5022, Time (s): 189.5833, Test accuracy: 0.5117 F1: 0.665142857142857


  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

  0%|          | 0/175 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Time/batch_time 195.000638128 1750
Time/sps 0.8998054944915187 1750
Epoch [10/10], Loss: 27.9675, Time (s): 195.0006, Test accuracy: 0.9383 F1: 0.9422776911076443
Mean time: 194.4865


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



------- Test/accuracy tensor(0.9250, device='cuda:0') 1750
------- Test accuracy: 0.9250
------- Test F1: 0.9258649093904449
